In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override = True)

True

In [58]:
%pip install --upgrade tiktoken


[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.8/254.8 kB 1.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
def load_document(file):
    from langchain.document_loaders import PyPDFLoader
    loader = PyPDFLoader(file)
    data = loader.load()
    return data

In [3]:
def chunk_data(data):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 256, chunk_overlap = 0)
    chunks = text_splitter.split_documents(data)
    return chunks

In [4]:
def embedding_cost(texts):
    import tiktoken
    encoding = tiktoken.encoding_for_model("text-embedding-ada-002")
    total_tokens = sum([len(encoding.encode(page.page_content)) for page in texts])
    print(f'Total tokens : {total_tokens}')
    print(f'{total_tokens / 1000 * 0.0004:.6f}')
    return total_tokens

In [5]:
def insert_of_fetch_embeddings(index_name):
    import pinecone
    from langchain.vectorstores import Pinecone
    from langchain.embeddings.openai import OpenAIEmbeddings

    embeddings = OpenAIEmbeddings()
    
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))

    if index_name in pinecone.list_indexes():
        print(f'Index {index_name} already exists. Loading embeddings ... ', end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        print(f'Creating index {index_name} and embeddings ...', end='')
        pinecone.create_index(index_name, dimension=1536, metric='cosine')
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')
        
    return vector_store

In [6]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))
    
    if index_name == 'all':
        indexes = pinecone.list_indexes()
        print('Deleting all indexes ... ')
        for index in indexes:
            pinecone.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pinecone.delete_index(index_name)
        print('Ok')

In [7]:
def ask_and_get_answer(vector_store, q):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    
    answer = chain.run(q)
    return answer
    
    
def ask_with_memory(vector_store, question, chat_history=[]):
    from langchain.chains import ConversationalRetrievalChain
    from langchain.chat_models import ChatOpenAI
    
    llm = ChatOpenAI(temperature=1)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
    
    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    result = crc({'question': question, 'chat_history': chat_history})
    chat_history.append((question, result['answer']))
    
    return result, chat_history

In [45]:
chunks = []
for i in os.listdir('data'):
    if i.endswith('.pdf'):
        data = load_document('data/' + i)
        chunks_ = chunk_data(data)
        chunks.extend(chunks_)

In [46]:
len(chunks)

8052

In [49]:
delete_pinecone_index(index_name='all')

Deleting all indexes ... 
Ok


In [8]:
embedding_cost(chunks)

NameError: name 'chunks' is not defined

In [8]:
pip install langchain


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
index_name = 'askadocument'
vector_store = insert_of_fetch_embeddings(index_name)

ModuleNotFoundError: No module named 'langchain'

In [3]:
q = "Tell me about S. Dillon Ripley Center"
answer = ask_and_get_answer(vector_store, q)
print(answer)

NameError: name 'vector_store' is not defined

In [63]:
# asking with memory
chat_history = []
while(5 > 4):
    question = input()
    result, chat_history = ask_with_memory(vector_store, question, chat_history)
    print('User: ' + question)
    print('SmithAI: ' + result['answer'])

User: heloo
SmithAI: Hello! How can I assist you today?
User: Tell me about S. Dillon Ripley Center and how can i get there
SmithAI: The S. Dillon Ripley Center is an underground building located on Jefferson Drive in Washington, DC. It is named after the Smithsonian's 8th Secretary and serves as the home for the Smithsonian Associates and the Discovery Theater. The center also connects underground to the African Art Museum. 

To get to the Ripley Center, you can access its copper domed entrance on Jefferson Drive between the "Castle" (the Smithsonian Castle) and the Freer Gallery of the National Museum of Asian Art.
User: Where was the Ancestral Connections exhibition located, in which museum?
SmithAI: The Ancestral Connections exhibition was located in the National Museum of the American Indian George Gustav Heye Center.
User: And where can i find the primates?
SmithAI: You can find primates at the Smithsonian's National Zoo in Washington, USA. They have over a dozen species of prima

KeyboardInterrupt: Interrupted by user